来自 https://github.com/eric-mitchell/direct-preference-optimization/blob/main/trainers.py

In [ ]:
import torch.nn.functional as F

In [ ]:
def dpo_loss(chosen_logps,
             rejected_logps,
             reference_chosen_logps,
             reference_rejected_logps,
             beta:float,
             label_smoothing:float=0.0,
             ipo:bool=False,
             reference_free:bool=False):
    
    pi_logratios = chosen_logps - rejected_logps 
    ref_logratios = reference_chosen_logps - reference_rejected_logps 
    chosen_rewards = beta * (chosen_logps - reference_chosen_logps)
    rejected_rewards = beta * (rejected_logps, reference_rejected_logps)

    if reference_free:
        ref_logratios = 0 
    logits = pi_logratios - ref_logratios 

    if ipo:
        losses = (logits - 1/(2*beta))**2
    else:
        losses = -(1-label_smoothing)*F.logsigmoid(beta*logits) - label_smoothing*F.logsigmoid(-beta*logits)

    return losses, chosen_rewards, rejected_rewards